In [12]:
import pandas as pd
import numpy as np
import yfinance as yf

def stock_data(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    fin = stock.financials
    q_fin = stock.quarterly_financials
    q_bs = stock.quarterly_balance_sheet

    try:
        rev_annual = fin.loc["Total Revenue"].sort_index(ascending=True)
        latest_rev = rev_annual.iloc[-1]
        y3to4_years_ago_rev = rev_annual.iloc[-4]
        days_elapsed = (pd.Timestamp.today() - pd.Timestamp(year=pd.Timestamp.today().year, month=1, day=1)).days
        n_years = 3 + days_elapsed / 365.25
        cagr = ((latest_rev / y3to4_years_ago_rev) ** (1/n_years) - 1) * 100
        growth_3to4y = ((latest_rev / y3to4_years_ago_rev) - 1) * 100
    except:
        cagr = growth_3to4y = 0

    try:
        ebit_ttm = q_fin.loc["EBIT"].iloc[:4].sum()
    except:
        ebit_ttm = None

    try:
        invested_capital = q_bs.loc["Invested Capital"].iloc[0]
    except:
        invested_capital = None

    pe = info.get('trailingPE')
    peg = info.get('trailingPegRatio')
    growth = (pe / peg) if peg and peg > 0 and pe else 0

    try:
        roic = ebit_ttm / invested_capital if ebit_ttm and invested_capital else None
    except:
        roic = None

    return {
        'short_name': info.get('shortName'),
        'ticker': ticker.upper(),
        'price': info.get('currentPrice'),
        'pe_ratio': pe,
        'peg_ratio': peg,
        'eps_ttm': info.get('trailingEps'),
        'Total Revenue': info.get('totalRevenue'),
        'growth': growth,
        '3-4 Year Sales Growth': growth_3to4y,
        'CAGR': cagr,
        'EBIT': ebit_ttm,
        'enterprise_value': info.get("enterpriseValue"),
        'invested_capital': invested_capital,
        "ROIC": roic,
        'dividendYield': info.get("dividendYield"),
        'dividend_yield%': info.get("trailingAnnualDividendYield"),
        'earnings_growth': info.get('earningsQuarterlyGrowth'),
        'roe': info.get('returnOnEquity'),
        'roa': info.get('returnOnAssets'),
        'debt_to_equity': (info.get('debtToEquity') or 0) * 0.01,
        'pb_ratio': info.get('priceToBook'),
        'free_cash_flow': info.get('freeCashflow'),
        'market_cap': info.get('marketCap'),
        'rev_growth': info.get("revenueGrowth"),
        'sector': info.get('sector')
    }

def safe_compare(val, threshold, comp='>='):
    if val is None:
        return False
    try:
        if comp == '>=':
            return val >= threshold
        elif comp == '<=':
            return val <= threshold
        elif comp == '>':
            return val > threshold
        elif comp == '<':
            return val < threshold
        elif comp == '==':
            return val == threshold
    except:
        return False
    return False

def peter_lynch(data):
    growth = data.get("growth")
    dividend_yield = data.get("dividend_yield%") or 0
    pe_ratio = data.get("pe_ratio")
    score = ((growth * 2 + dividend_yield) / pe_ratio) if growth and pe_ratio else 0
    return round(score, 2)

def warren_buffett(data):
    score = 0
    if safe_compare(data.get("roe"), 0.12): score += 30
    if safe_compare(data.get("debt_to_equity"), 1.0, '<='): score += 20
    if safe_compare(data.get("pe_ratio"), 0, '>') and safe_compare(data.get("pe_ratio"), 30, '<'): score += 30
    if safe_compare(data.get("earnings_growth"), 0, '>'): score += 20
    return score

def philip_fisher(data):
    score = 0
    if safe_compare(data.get("earnings_growth"), 0.1): score += 35
    if safe_compare(data.get("rev_growth"), 0.1, '>'): score += 35
    if safe_compare(data.get("roe"), 0.15, '>'): score += 30
    return score

def magic_formula(data):
    score = 0
    try:
        earn_yield = data['EBIT'] / data['enterprise_value']
    except:
        earn_yield = None
    if safe_compare(earn_yield, 0.07, '>'): score += 50
    if safe_compare(data.get("ROIC"), 0.1, '>'): score += 50
    return score

def seth_klarman(data):
    score = 0
    if safe_compare(data.get("pb_ratio"), 2, '<='): score += 34
    if safe_compare(data.get("free_cash_flow"), 0, '>'): score += 33
    if safe_compare(data.get("debt_to_equity"), 0.5, '<='): score += 33
    return score

def benjamin_graham(data):
    score = 0
    if safe_compare(data.get("pe_ratio"), 20, '<='): score += 20
    if safe_compare(data.get("pb_ratio"), 3, '<='): score += 20
    if safe_compare(data.get("debt_to_equity"), 1.0, '<='): score += 20
    if safe_compare(data.get("dividend_yield%"), 0, '>'): score += 20
    if safe_compare(data.get("earnings_growth"), 0, '>'): score += 20
    return score

def personal_model(data):
    score = 0
    if safe_compare(data.get("roe"), 0.14): score += 12.5
    if safe_compare(data.get("ROIC"), 0.10): score += 12.5
    if safe_compare(data.get("earnings_growth"), 0.12): score += 12.5
    if safe_compare(data.get("growth"), 0.10): score += 12.5
    if safe_compare(data.get("pe_ratio"), 40, '<='): score += 12.5
    if safe_compare(data.get("peg_ratio"), 2.5, '<'): score += 12.5
    if safe_compare(data.get("debt_to_equity"), 0.5, '<='): score += 12.5
    if safe_compare(data.get("free_cash_flow"), 0, '>'): score += 12.5
    return score

def stock_df(tickers):
    rows = []
    for ticker in tickers:
        data = stock_data(ticker)
        row = {k.replace('_', ' ').title(): v for k, v in data.items()}
        rows.append(row)
    return pd.DataFrame(rows).round(2)

def method_df(tickers):
    rows = []
    for ticker in tickers:
        data = stock_data(ticker)
        pl_score = peter_lynch(data)
        scores = [
            warren_buffett(data),
            philip_fisher(data),
            magic_formula(data),
            benjamin_graham(data),
            seth_klarman(data),
            personal_model(data)
        ]
        avg_score = np.mean(scores)

        if 0.5 <= pl_score < 1: pl_adj = 20
        elif 1 <= pl_score < 1.5: pl_adj = 50
        elif 1.5 <= pl_score < 2: pl_adj = 75
        elif pl_score >= 2: pl_adj = 100
        else: pl_adj = 0

        final_score = round((avg_score * 6 + pl_adj) / 7, 2)

        row = {
            "Ticker": data["ticker"],
            "Peter Lynch": pl_score,
            "Warren Buffett": scores[0],
            "Philip Fisher": scores[1],
            "Magic Formula": scores[2],
            "Benjamin Graham": scores[3],
            "Seth Klarman": scores[4],
            "Personal Model": scores[5],
            "Final Score": f"{final_score}%"
        }
        rows.append(row)
    return pd.DataFrame(rows).round(2)


In [10]:
stock_df([
    "AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "NFLX"
]
)

,Short Name,Ticker,Price,Pe Ratio,Peg Ratio,Eps Ttm,Total Revenue,Growth,3-4 Year Sales Growth,Cagr,...,Dividend Yield%,Earnings Growth,Roe,Roa,Debt To Equity,Pb Ratio,Free Cash Flow,Market Cap,Rev Growth,Sector
0,Apple Inc.,AAPL,213.76,33.30,2.00,6.42,400366010368,16.68,6.89,1.89,...,0.00,0.05,1.38,0.24,1.47,47.81,97251500032,3192676417536,0.05,Technology
1,Microsoft Corporation,MSFT,510.88,39.45,2.37,12.95,270010007552,16.67,45.83,11.18,...,0.01,0.18,0.34,0.15,0.33,11.80,54817001472,3797136179200,0.13,Technology
2,Alphabet Inc.,GOOGL,192.17,20.49,1.42,9.38,359713013760,14.40,35.86,8.99,...,0.00,0.46,0.35,0.17,0.08,7.22,60679000064,2338997141504,0.12,Communication Services
3,"Amazon.com, Inc.",AMZN,232.23,37.82,2.97,6.14,650313007104,12.74,35.79,8.97,...,0.00,0.64,0.25,0.08,0.52,8.06,39273500672,2465446494208,0.09,Consumer Cyclical
4,"Tesla, Inc.",TSLA,305.30,173.47,5.82,1.76,95724003328,29.83,81.50,18.22,...,0.00,-0.71,0.09,0.04,0.17,13.47,3360000000,983359029248,-0.09,Consumer Cyclical
5,"Meta Platforms, Inc.",META,714.80,27.98,2.36,25.55,170359996416,11.87,39.49,9.80,...,0.00,0.35,0.40,0.18,0.27,9.75,36658999296,1797243142144,0.16,Communication Services
6,NVIDIA Corporation,NVDA,173.74,55.86,1.66,3.11,148514996224,33.74,384.87,55.78,...,0.00,0.26,1.15,0.53,0.12,50.54,55443001344,4237101891584,0.69,Technology
7,"Netflix, Inc.",NFLX,1180.76,50.42,2.17,23.42,41693184000,23.20,31.33,7.95,...,0.00,0.46,0.44,0.15,0.68,20.11,22694656000,501735653376,0.16,Communication Services


In [15]:
method_df(["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "NVDA", "NFLX"])

,Ticker,Peter Lynch,Warren Buffett,Philip Fisher,Magic Formula,Benjamin Graham,Seth Klarman,Personal Model,Final Score
0,AAPL,1.00,50,30,50,40,33,75.0,46.86%
1,MSFT,0.85,70,100,50,60,66,100.0,66.57%
2,GOOGL,1.41,100,100,50,40,66,100.0,72.29%
3,AMZN,0.67,70,65,50,40,33,75.0,50.43%
4,TSLA,0.34,20,0,50,20,66,50.0,29.43%
5,META,0.85,100,100,50,60,66,100.0,70.86%
6,NVDA,1.21,70,100,50,60,66,87.5,69.07%
7,NFLX,0.92,70,100,50,40,33,75.0,55.43%
